# Precog Recruitment Task 2

## Object Detection

#### Dependencies (sourced cited at the bottom where required)

In [ ]:
import torch
import os
import shutil
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
img_id=input()
img_path = f"data/img/{img_id}.png"
# Check if the input image exists
if not os.path.exists(img_path):
    print("Input image not found.")
    raise(NameError)

# copying image into new directory for easy access 
file_dir="curdir"
if os.path.exists(file_dir):
    shutil.rmtree(file_dir)
os.makedirs(file_dir)

# Define the output image path
output_image_path = os.path.join(file_dir, "caption.jpg")
# Copy the input image to the output folder with the specified name
shutil.copy(img_path, output_image_path)



### Caption Removal (Inpainting) 

In [ ]:

import keras_ocr
# Function to calculate midpoint of a line
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return (x_mid, y_mid)


# Initialize keras-ocr pipeline
pipeline = keras_ocr.pipeline.Pipeline()


image_path = 'curdir/caption.jpg'

# Read the image
image = keras_ocr.tools.read(image_path) 

# Recognize text in the image
predictions = pipeline.recognize([image])

# Create a mask for inpainting
mask = np.zeros(image.shape[:2], dtype="uint8")

# Iterate through predicted text regions and create mask
for box in predictions[0]:
    x0, y0 = box[1][0]
    x1, y1 = box[1][1] 
    x2, y2 = box[1][2]
    x3, y3 = box[1][3]
    
    # Calculate midpoints for line drawing
    x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
    x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
    
    # Calculate thickness based on line length
    thickness = int(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))
    
    # Draw line on mask
    cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)

# Inpaint the text regions
inpainted_image = cv2.inpaint(image, mask, 7, cv2.INPAINT_NS)


# Save the image without text
cv2.imwrite('curdir/nocaption.jpg', cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB))

### Text Extraction

The text is extracted from the dataset as it is provided, but text extraction using OCR is implemented in the text_recognition for more general applications.

In [ ]:
import jsonlines

#image text can be in any of the 3 files
file_paths = ['data/train.jsonl', 'data/test.jsonl','data/dev.jsonl'] 

given_text = None
#function to search and extract text
for file_path in file_paths:
        with jsonlines.open(file_path) as reader:
            for obj in reader:
                # print(obj["id"])
                
                if obj["id"] == int(img_id):
                    
                    given_text=obj["text"]

print(given_text)

### Object Detection
Yolov8 from the ultralytics module was used as its pretrained model gave the highest accuracy

In [ ]:
from ultralytics import YOLO
#m indicates medium model . medium showed best results . other availble models yolov8n.pt yolov8s.pt yolov8m.pt yolov8l.pt yolov8x.pt
model = YOLO('yolov8m.pt')

#### Object Detection with Captions

In [ ]:
import matplotlib.image as mpimg

#saves to a specfic folder , conf set low to account for caption obstruction
model(source="curdir/caption.jpg", show=False, conf=0.5, save=True,project='curdir', name='captions')

image = mpimg.imread("curdir/captions/caption.jpg")

# Display the image
plt.imshow(image)
plt.axis('off')  # Turn off axis
plt.show()

#### Object Detection without caption


In [ ]:

#saves to a specfic folder 
predictions = model(source="curdir/nocaption.jpg", show=False, conf=0.6, save=True,project='curdir', name='no_captions',save_txt=None)

#dataset labels , present in the output , hardcoded for easier access
labels={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

# print(predictions)

labels_seen=[]
prompt_caption_gen=" "
for idx, prediction in enumerate(predictions[0].boxes.xywhn): # change final attribute to desired box format
    cls = int(predictions[0].boxes.cls[idx].item())
    # Write line to file in YOLO label format : cls x y w h
    label=labels[cls]
    
    # Adding all labels except 0 (person) , person will be detected earlier , testing showed repeats reduce accuracy of caption model hence removed
    if cls!=0 and cls not in labels_seen:
        prompt_caption_gen+=label+" "
    labels_seen.append(cls)
    
          
        
          
        
image = mpimg.imread("curdir/no_captions/nocaption.jpg")

plt.imshow(image)
plt.axis('off')  # Turn off axis
plt.show()

### Race Detection
using DeepFace module , in paticular the gender and race and emotion detector. The model also supports detection of age and emotion

In [ ]:
from deepface import DeepFace

# using deepface , enforce_detection set to False as some memes might not have people , age can be added as a action
analysis = DeepFace.analyze(img_path, actions = ["gender", "race","emotion"],enforce_detection=False)
print(analysis)
tags_seen=[]

# potential add emotion as well 
for person in analysis:
    # not adding repeats
    if person["dominant_gender"] not in tags_seen and person["dominant_race"] not in tags_seen : 
        prompt_caption_gen += person["dominant_race"] + " " + person["dominant_gender"] + " " 
        tags_seen.append(person["dominant_race"])
        tags_seen.append(person["dominant_gender"])

    
print(prompt_caption_gen)


### Caption generation

In [ ]:

from promptcap import PromptCap

# using promptcap to generate the caption
model = PromptCap("tifa-benchmark/promptcap-coco-vqa")  

image = "curdir/nocaption.jpg"
caption_generated=model.caption(prompt_caption_gen, image)
print(caption_generated)

### Sentence Similarity using semantic analysis

Here, sentence similarity is used as the measure to determine whether an image is a meme or not. The two text fields are:

1. The given text in the image.
2. Caption generated based on the images, generated with the context of races and objects within the image.


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model= SentenceTransformer('bert-base-nli-mean-tokens')

#set of resultant vectors for each sentence
vector=model.encode([given_text,caption_generated])

# current implementation uses only two sentences hence 1:
print(cosine_similarity([vector[0]],vector[1:]))

### Hateful Meme Classifier Based on text

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# using pretrained BERT for sentiment analysis
#https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
tokens = tokenizer.encode(given_text, return_tensors='pt')
result = model(tokens)
score= int(torch.argmax(result.logits))+1
print(score)


